# SENTIMENT ANALYSIS WITH FINGPT
[FinGPT Github](https://github.com/AI4Finance-Foundation/FinGPT?tab=readme-ov-file)


### Problems
- Need GPU; ram explode in colab

## Packages

In [ ]:
!pip install datasets
!pip install peft
!pip install rouge_score

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/sentiment')

import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from utils.FinGPT_Forecaster_utils import *

### FinGPT Intro
1. Based on Llama\
😢 Need GPU\
😢 Run on Colab: ram explode

2. Errors\
  (1) 👌[Solved] [Tokens - fast tokens]( https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/5): [Fast Tokens Elaboration](https://github.com/huggingface/transformers/releases/tag/v4.0.0)\
  (2) Ram explode...

In [ ]:
!pip uninstall -y transformers
!pip install transformers[sentencepiece]
!pip install accelerate
!pip install bitsandbytes

In [ ]:
# need transformers >= 4.36
import transformers
transformers.__version__

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0

# Load Models
base_model = "NousResearch/Llama-2-13b-hf"
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = False,)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]

# Generate results
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512)
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

# Output:
# positive
# neutral
# negative

## FinGPT - Forecaster
1. Based on Llama
2. Need the permission to download models: need apply again
3. Need GPU

😢 Waiting for permission of the access of Llama 2 \
⛄ [Access to Llama 2 on Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf)\
⛄ [Github Forecaster](https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Forecaster)

In [29]:
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
)
base_model.model_parellal = True

OSError: ignored

In [ ]:
model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
test_dataset = load_from_disk('data/fingpt-forecaster-dow30v3-20221231-20230531-llama/')['test']

In [ ]:
def test_demo(model, tokenizer, prompt):

    inputs = tokenizer(
        prompt, return_tensors='pt',
        padding=False, max_length=4096
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output
    # return res

In [ ]:
answers, gts = [], []

for i in range(len(test_dataset)):
    prompt = test_dataset[i]['prompt']
    output = test_demo(model, tokenizer, prompt)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)
    gt = test_dataset[i]['answer']
    print('\n------- Prompt ------\n')
    print(prompt)
    print('\n------- LLaMA2 Finetuned ------\n')
    print(answer)
    print('\n------- GPT4 Groundtruth ------\n')
    print(gt)
    print('\n===============\n')
    answers.append(answer)
    gts.append(gt)

In [ ]:
gts[0], answers[0]

In [ ]:
calc_metrics(answers, gts)